## Sentinel-1 data download using sentinelsat

In [10]:
# import required packages
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
import os

def download_sentinel(platform, prod_type, scihub_user, scihub_pass, region, start_date, end_date, down_dir=None):
    # change the working directory to the location of files
    if down_dir!=None:
        os.chdir(down_dir)
    
    # connect to the API
    api = SentinelAPI(scihub_user, scihub_pass, 'https://tmphub.copernicus.eu/dhus/')#'https://scihub.copernicus.eu/dhus')
    
    # search by polygon, time, and Hub query keywords
    ## TO-DO: add type of product to the search terms
    footprint = geojson_to_wkt(read_geojson(region))
    
    products = api.query(footprint,
                         date = (start_date, end_date),
                         producttype = prod_type,
                         platformname = platform)
    
    # download all results from the search
    print("Files will be downloaded to {}".format(os.getcwd()))
    api.download_all(products)
    

Definition of the environment variables and settings for downloads 

In [8]:
# Get GeoJSON file with region extent
Ibague_ext ="/mnt/workspace_cluster_6/TRANSVERSAL_PROJECTS/MADR/COMPONENTE_2/Imagenes_Satelitales/Sentinel_2/JSON_Ibague/IbagueJSON.geojson"
# Set download directory
downdir = '/home/azalazar/data'
# Set ESA scihub credentials
scihub_user = 'asalazarr'
scihub_pass = 'tila8sude'

Download of Sentinel-1 and Sentinel-2 from march 2018 images for Ibague region

In [11]:
#download_sentinel('Sentinel-1', 'GRD', scihub_user, scihub_pass, Ibague_ext, "20180328", "20180331", downdir)
download_sentinel('Sentinel-2', 'S2MSI1C', scihub_user, scihub_pass, Ibague_ext, "20180328", "20180331", downdir)

SentinelAPIError: HTTP status 401 Unauthorized: 
# HTTP Status 401 - Full authentication is required to access this resource

 **type** Status report

 **message** _Full authentication is required to access this resource_

 **description** _This request requires HTTP authentication._

* * *

### Apache Tomcat/8.0.36

In [2]:
# Unzip files
import zipfile

def unzip_eofiles(eo_dir, unzip_dir = 'uz_data'):
    # List all zip files in directory
    eo_files = filter(re.compile('zip$').search, os.listdir(eo_dir))
    
    # Check if a data folder exist
    if not os.path.exists(unzip_dir):
        os.makedirs(unzip_dir)
        print 'data folder' + ' was created'
    
    ## Loop over list of zip files
    for im_id in eo_files:
        if not os.path.exists('data/'+im_id[:-3]+'SAFE'):
            print('Unzipping ' + im_id)
        zip_ref = zipfile.ZipFile(im_id, 'r')
        zip_ref.extractall('data')
        zip_ref.close()
    else:
        print(im_id[:-4] + ' was already uncompressed')


## Pre-processing using SNAP
The pre-processing workflow (to-be revised) is performed using SNAP Python API, snappy, and currently incudes the following steps:
1. Apply orbit
2. Speckle filtering
3. Terrain correction
4. Subset the area of interest
5. Logaritmic transformation (to dB)
6. Texture analysis

***
Help functions:

In [18]:
## Get a list of all Sentinel-1 toolbox operators
op_spi = snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpis().toString()
op_list = op_spi.split(', ')
listname = []
for op_str in op_list:
    to_add = op_str
    if op_str[0] == '[': to_add = op_str[1:]
    elif op_str[-1] == ']': to_add = op_str[:-1]
    #if to_add.split('.')[2] == 's1tbx':
    listname.append(to_add.split('$')[0])
listname.sort()
#for name in listname:
#    print(name)

In [6]:
## Get help of the SNAP operators
def Op_help(op):
        op_spi = snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(op)
        print('Op name: {}'.format(op_spi.getOperatorDescriptor().getName()))
        print('Op alias: {}\n'.format(op_spi.getOperatorDescriptor().getAlias()))
        print('PARAMETERS:\n')
        param_Desc = op_spi.getOperatorDescriptor().getParameterDescriptors()
        for param in param_Desc:
            value_set = param_Desc[0].getValueSet()
            if len(value_set) == 0:
                print('{}: {}\nDefault Value: {}\n'.format(param.getName(),param.getDescription(),param.getDefaultValue()))
            else:
                print('{}: {}\nDefault Value: {}\nPossible param: {}\n'.format(param.getName(),param.getDescription(),param.getDefaultValue(),list(value_set)))

#Op_help("Multi-Temporal-Speckle-Filter")
Op_help("Resample")

Op name: org.esa.snap.core.gpf.common.resample.ResamplingOp
Op alias: Resample

PARAMETERS:

referenceBandName: The name of the reference band. All other bands will be re-sampled to match its size and resolution. Either this or targetResolutionor targetWidth and targetHeight must be set.
Default Value: None

targetWidth: The width that all bands of the target product shall have. If this is set, targetHeight must be set, too. Either this and targetHeight or referenceBand or targetResolution must be set.
Default Value: None

targetHeight: The height that all bands of the target product shall have. If this is set, targetWidth must be set, too. Either this and targetWidth or referenceBand or targetResolution must be set.
Default Value: None

targetResolution: The resolution that all bands of the target product shall have. The same value will be applied to scale image widths and heights. Either this or referenceBand or targetwidth and targetHeight must be set.
Default Value: None

upsamplin

### Read and process S1 products

The following code reads directories in a folder containing only S1 products. Then, creates a dictionary using the products names as keys. Then, reads and stores diferent processing steps in a second-level dictionary (i.e. inside the first dictionary). 

In [5]:
import snappy, shutil, os, ast, re
from sentinelsat.sentinel import read_geojson, geojson_to_wkt
from snappy import ProductIO, HashMap, GPF, jpy

GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
HashMap = snappy.jpy.get_type('java.util.HashMap')
WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')

In [37]:
def getBandNames (product, sfilter = ''):
    """
    Produces a string to use in the sourceBandNames parameter specification of SNAP operators.
    Args:
        product (): 
        sfilter (string): regular expression to filter the name of the bands
    Output:
        returns a string with comma-separated band names
    """
    band_names = product.getBandNames()
    if sfilter != '':
        band_names = filter(re.compile(r''+sfilter).search, band_names)
    if len(band_names) > 0:
        band_names = ','.join(band_names)
    else:
        band_names = None
    return band_names

def stacking(product_set):
    """
    Takes a list of SNAP products and returns a stacked product with all the bands named with
    the products acquisition dates.
    Args:
        product_set: a list of products to be stacked
    Output: returns an individual product with the bands of the other products 
    """
    # check if products contain any bands, discard when not
    prod_set = [product for product in product_set if not product.getNumBands() == 0]
    # define the stack parameters
    params = HashMap()
    params.put('resamplingType', None)
    params.put('initialOffsetMethod', 'Product Geolocation')
    params.put('extent', 'Master')
    # create the stack
    print("Creating stack of {} products...".format(str(len(prod_set)))
    create_stack = GPF.createProduct('CreateStack', params, prod_set)
    return create_stack

def mtspeckle_sigma0 (stacked_prod, pol):
    """
    Applies the a multi-temporal speckle filter to the a corregistered calibrated product stack. Takes the product bands
    which name starts with 'Sigma0'.
    
    Args:
        stacked_prod (): product with all the bands to be used for the multi-temporal speckle filter operation
        pol (str): polarization to apply the speckle filter (VV or VH)
    Output:
    """
    param_specklefilter = HashMap()
    param_specklefilter.put('sourceBandNames', getBandNames(stacked_prod, "Sigma0_"+pol))
    param_specklefilter.put('filter', 'Lee Sigma')
    sf_product = GPF.createProduct("Multi-Temporal-Speckle-Filter", param_specklefilter, stacked_prod)
    return sf_product

def Sigma0_todB (product):
    """
    Transforms the product bands to a logaritmic scale in dB (10*log10[band]).
    
    Args:
        product: product with Sigma0 bands in linear units
    Output:
    """
    param_logdB = HashMap()
    param_logdB.put('sourceBandNames', getBandNames(product))
    db_product = GPF.createProduct("LinearToFromdB", param_logdB, product)
    return db_product

def write_product (product, out_name):
    """
    Writes a GDF product in BEAM-DIMAP format (.dim). Prints informative text with product name and
    names of the bands.
    
    Args:
        product (): product to be written
        out_name (str): name/location of the output file
    """
    print('Writing {}, with bands: {}.'.format(out_name, getBandNames(product)))
    ProductIO.writeProduct(product, out_name, 'BEAM-DIMAP', pm = createProgressMonitor())

def createProgressMonitor():
    PWPM = jpy.get_type('com.bc.ceres.core.PrintWriterProgressMonitor')
    JavaSystem = jpy.get_type('java.lang.System')
    monitor = PWPM(JavaSystem.out)
    return monitor

* Set processing variables: data location, polarizations and region of interest

In [40]:
# Set a variable for the data location
eo_direc = 'D:/eo_data/Saldana/'

# Get the names of S1 files in location
eo_files = filter(re.compile(r'S1A.*SAFE$').search, os.listdir(eo_direc))

## Create a dictionary to read Sentinel-1 L1 GRD products
product = {}
for element in eo_files:
    # second level dictionary to store intermediate products in memory
    product[element[:-5]] = {}

# Read the area of interest from geoJSON file
regPath = "//dapadfs/workspace_cluster_6/TRANSVERSAL_PROJECTS/MADR/COMPONENTE_2/" \
    "Imagenes_Satelitales/Sentinel_2/JSON_Saldana/saldanaextent.geojson"
geom = geojson_to_wkt(read_geojson(regPath))

# Define polarizations of interest
polarizations = ['VV', 'VH']

# Set location for output files
pre_outdir = eo_direc + 'prep/'
if not os.path.exists(pre_outdir):
    os.makedirs(pre_outdir)
    print "New directory '" + pre_outdir + "' was created"


* Apply orbit file
* Radiometric calibration
* Terrain correction
* Subset
* Write intermediate results

In [42]:
for key, value in product.iteritems():    
    # Read the product
    value['GRD'] = ProductIO.readProduct(eo_direc+key+'.SAFE/manifest.safe')
    print('Reading '+key)
    
    # Apply orbit
    param_orbit = HashMap()
    value['orbit'] = GPF.createProduct("Apply-Orbit-File", param_orbit, value['GRD'])
    
    # The following operations are specific por each polarization    
    for pol in polarizations:
        # Radiometric calibration
        param_calibration = HashMap()
        param_calibration.put('outputSigmaBand', True)
        param_calibration.put('sourceBands', getBandNames(value['orbit'], 'Intensity_'+pol))
        param_calibration.put('selectedPolarisations', pol)
        param_calibration.put('outputImageScaleInDb', False)
        value['calibration_'+pol] = GPF.createProduct("Calibration", param_calibration, value['orbit'])
        
        # Terrain correction
        param_terraincor = HashMap()
        param_terraincor.put('demResamplingMethod', 'NEAREST_NEIGHBOUR')
        param_terraincor.put('imgResamplingMethod', 'NEAREST_NEIGHBOUR')
        param_terraincor.put('applyRadiometricNormalization', True)
        param_terraincor.put('demName', 'SRTM 3Sec')
        param_terraincor.put('pixelSpacingInMeter', 10.0)
        param_terraincor.put('sourceBands', getBandNames(value['calibration_'+pol], 'Sigma0_'+pol))
        param_terraincor.put('mapProjection', 'WGS84(DD)')
        value['terraincor_'+pol] = GPF.createProduct("Terrain-Correction", param_terraincor, value['calibration_'+pol])
        
        # Subset to area of interest
        param_subset = HashMap()
        param_subset.put('geoRegion', geom)
        param_subset.put('outputImageScaleInDb', False)
        param_subset.put('sourceBandNames', getBandNames(value['terraincor_'+pol], 'Sigma0_'+pol))
        value['subset_'+pol] = GPF.createProduct("Subset", param_subset, value['terraincor_'+pol])
        
        # define the name of the output
        output_name = pre_outdir + pol + "_" + key
        
        # Write the results to files
        write_product(value['subset_'+pol], output_name)
                
        # dispose all the intermediate products
        value['calibration_'+pol].dispose()
        value['terraincor_'+pol].dispose()
        value['subset_'+pol].dispose()
        
    #dispose all the intermediate products
    value['GRD'].dispose()
    value['orbit'].dispose()


Reading S1A_IW_GRDH_1SSV_20151017T104253
Writing D:/eo_data/Saldana/prep/VV_S1A_IW_GRDH_1SSV_20151017T104253, with bands: Sigma0_VV.
Writing D:/eo_data/Saldana/prep/VH_S1A_IW_GRDH_1SSV_20151017T104253, with bands: None.
Reading S1A_IW_GRDH_1SSV_20160419T231329
Writing D:/eo_data/Saldana/prep/VV_S1A_IW_GRDH_1SSV_20160419T231329, with bands: Sigma0_VV.


KeyboardInterrupt: 

### Speckle filtering 

* Read and stack pre-processed products by polarization
* Multi-temporal speckle filtering
* Scale transformation to dB

In [36]:
int_files = os.listdir(eo_direc+'prep/')

polarizations = ['VV','VH']

## Make stack of polarizations, apply mt speckle filter, log trasnform and write
for pol in polarizations:
    # filter results by polarization
    results = filter(re.compile(r'^'+pol+'.*dim$').search, eo_files)
    # declare variable to read products in list
    polprods = []
    # read product
    for result in results:
        polprods.append(ProductIO.readProduct(eo_direc+'prep/'+result))
    # stack, apply multi-temporal speckle filter and logaritmic transform
    stack = Sigma0_todB(mtspeckle_sigma0(stacking(polprods), pol))
    # write results
    write_product(stack, pre_outdir+pol+'stack_spk_dB')
    # memory cleaning
    stack.dispose()
    polprods = []


Creating stack...
Writing , with bands: Sigma0_VV_mst_19Apr2016_db,Sigma0_VV_slv1_02Apr2016_db,Sigma0_VV_slv2_23Aug2015_db,Sigma0_VV_slv3_09Mar2016_db,Sigma0_VV_slv4_30Aug2015_db,Sigma0_VV_slv5_02Mar2016_db,Sigma0_VV_slv6_30Jul2015_db,Sigma0_VV_slv7_06Aug2015_db,Sigma0_VV_slv8_19Jun2015_db,Sigma0_VV_slv9_13Jul2015_db,Sigma0_VV_slv10_10Oct2015_db,Sigma0_VV_slv11_04Dec2015_db,Sigma0_VV_slv12_10Nov2015_db,Sigma0_VV_slv13_03Nov2015_db,Sigma0_VV_slv14_17Oct2015_db,Sigma0_VV_slv15_07Feb2016_db,Sigma0_VV_slv16_21Jan2016_db,Sigma0_VV_slv17_14Feb2016_db,Sigma0_VV_slv18_14Jan2016_db,Sigma0_VV_slv19_21Dec2015_db,Sigma0_VV_slv20_27Nov2015_db,Sigma0_VV_slv21_12Jun2015_db,Sigma0_VV_slv22_16Sep2015_db.
Creating stack...
Writing , with bands: Sigma0_VH_mst_27Nov2015_db,Sigma0_VH_slv1_16Sep2015_db.


### Texture analysis

In [18]:
# paramaters for GLCM texture analysis
paramGLCM = HashMap()
paramGLCM.put('sourceBandNames', 'Sigma0_' + 'VV')
paramGLCM.put('windowSizeStr', '5x5')
paramGLCM.put('quantizerStr', 'Probabilistic Quantizer')
paramGLCM.put('quantizationLevelsStr', '16')
paramGLCM.put('displacement','4' )
paramGLCM.put('outputContrast','true')
paramGLCM.put('outputDissimilarity','true')
paramGLCM.put('outputHomogeneity','true')
paramGLCM.put('outputASM','true')
paramGLCM.put('outputEnergy','true')
paramGLCM.put('outputMean','true')
paramGLCM.put('outputVariance','true')
paramGLCM.put('outputCorrelation','true')
product = GPF.createProduct("GLCM", paramGLCM, product)